In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-acceptability-classification-dataset/car.csv
/kaggle/input/car-acceptability-classification-dataset/car.data


In [2]:
cars=pd.read_csv("/kaggle/input/car-acceptability-classification-dataset/car.csv")

In [3]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Buying_Price       1728 non-null   object
 1   Maintenance_Price  1728 non-null   object
 2   No_of_Doors        1728 non-null   object
 3   Person_Capacity    1728 non-null   object
 4   Size_of_Luggage    1728 non-null   object
 5   Safety             1728 non-null   object
 6   Car_Acceptability  1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [4]:
X=cars.drop("Car_Acceptability",axis=1)
y=cars.Car_Acceptability

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=50,random_state=42)

In [6]:
X_train.head()

,Buying_Price,Maintenance_Price,No_of_Doors,Person_Capacity,Size_of_Luggage,Safety
188,vhigh,high,4,more,big,high
818,high,low,4,2,big,high
943,med,vhigh,4,more,big,med
979,med,high,2,2,big,med
481,high,vhigh,3,more,med,med


In [7]:
for key in cars.keys():
    print(key,np.unique(cars[key]))

Buying_Price ['high' 'low' 'med' 'vhigh']
Maintenance_Price ['high' 'low' 'med' 'vhigh']
No_of_Doors ['2' '3' '4' '5more']
Person_Capacity ['2' '4' 'more']
Size_of_Luggage ['big' 'med' 'small']
Safety ['high' 'low' 'med']
Car_Acceptability ['acc' 'good' 'unacc' 'vgood']


In [8]:
def transform_data(X):
    if not isinstance(X,pd.DataFrame):
        X.to_frame()
    X_transformed=pd.DataFrame()
    for key in X.keys():
        key1_hot=pd.get_dummies(X[key])
        for key1 in key1_hot.keys():
            X_transformed[key+'_'+key1]=key1_hot[key1]
    return X_transformed

In [9]:
X_train_transformed=transform_data(X_train)

In [10]:
y_train_transformed=transform_data(y_train.to_frame())

In [11]:
y_train_transformed.keys()

Index(['Car_Acceptability_acc', 'Car_Acceptability_good',
       'Car_Acceptability_unacc', 'Car_Acceptability_vgood'],
      dtype='object')

For Purpose of EDA, We do binary analysis to try and observe what separates acceptable and Unacceptable cars. However, we will be doing Multiclass Classification only.

In [12]:
y_train_eda=pd.DataFrame()
y_train_eda["Acceptability"]=y_train_transformed["Car_Acceptability_unacc"].replace({0:1, 1:0})

In [13]:
X_train_eda=X_train_transformed
X_train_eda["Acceptability"]=y_train_eda["Acceptability"]

In [14]:
for key in X_train_transformed.keys():
    correlation=X_train_eda[key].corr(X_train_eda["Acceptability"])
    if not pd.isna(correlation):
        print(key,correlation)

Buying_Price_high -0.053602345387876005
Buying_Price_low 0.12504604698979455
Buying_Price_med 0.09800472702141917
Buying_Price_vhigh -0.16883504310087313
Maintenance_Price_high -0.03854732135854151
Maintenance_Price_low 0.11394592911076502
Maintenance_Price_med 0.09711458071620542
Maintenance_Price_vhigh -0.17183415600564297
No_of_Doors_2 -0.06902304750030226
No_of_Doors_3 0.004812092292694023
No_of_Doors_4 0.03487423252776662
No_of_Doors_5more 0.029540149946260107
Person_Capacity_2 -0.4586561750495898
Person_Capacity_4 0.24058448543408315
Person_Capacity_more 0.21670266382921394
Size_of_Luggage_big 0.10526486942943021
Size_of_Luggage_med 0.020709810356755204
Size_of_Luggage_small -0.1260496825301895
Safety_high 0.35055354327095983
Safety_low -0.4667059201952102
Safety_med 0.1189518878113959
Acceptability 0.9999999999999999


In [15]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    #loss_function='CrossEntropy'
)
clf.fit(X_train, y_train, 
        cat_features=range(0,6), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{}


In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,clf.predict(X_test))

1.0